# 动态规划算法
这部分算法包含两部分：策略迭代算法和价值迭代算法。

## 策略迭代算法
### 第一步：状态价值评估
从状态价值最优Bellman方程出发：

$V^{\pi}(s)=\sum_{a \in A}\pi(a|s)\left(r(s, a)+\gamma \sum_{s' \in S}P(s'|s, a)V^{\pi}(s)\right)$

可以将策略价值函数改写为一个递归的形式：

$V^{k+1}(s)=\sum_{a \in A}\pi(a|s)\left(r(s, a)+\gamma \sum_{s' \in S}P(s'|s, a)V^{k}(s)\right)$

其中$k$表示经过$k$次迭代的策略价值函数。

### 第二步：策略提升

策略提升定理：$Q^{\pi}(s, \pi' (s))\geq V^{\pi}(s)$，可以说明策略$\pi'$更优。

用贪心的方法确定新的策略：
$\pi' (s) = \text{argmax}_a Q^{\pi}(s, a) = \text{argmax}_a \left(r(s, a)+\gamma \sum_{s' \in S}P(s'|s, a)V^{k}(s)\right) $

In [1]:
import numpy as np

# 构建悬崖行走的环境
class CliffWalkingEnv:
    def __init__(self, nrow=4, ncolumn=12):
        # 总共具有的位置为nrow、ncolumn，每个位置存储动作数组，每个动作存储可能的到达的状态，状态包含四个信息（概率、新状态、奖励、done）
        self.nrow = nrow        #定义行
        self.ncolumn = ncolumn  #定义列
        self.P = self.createP()
    
    def createP(self):
        action = [[0, -1], [1, 0], [0, 1], [-1, 0]]
        P = np.zeros([self.nrow*self.ncolumn, len(action), 4])
        for i in range(self.nrow):
            for j in range(self.ncolumn):
                for l, _ in enumerate(action):
                    done = 0
                    reward = 0
                    new_row = min(self.nrow-1, max(0, i+action[l][0]))
                    new_column = min(self.ncolumn-1, max(0, j+action[l][1]))
                    #设置终止位置
                    if i == self.nrow-1 and j>0:
                        done = 1
                        reward = 0
                        P[i*self.ncolumn + j, l] = [1, i*self.ncolumn + j, reward, done]
                        continue
                    #设置奖励
                    if new_row == self.nrow-1 and new_column == self.ncolumn-1:
                        reward = 1
                    state = new_row*self.ncolumn + new_column
                    P[i*self.ncolumn + j, l] = [1, state, reward, done]
        return P

In [ ]:
#策略价值函数迭代计算
class PolicyIteration:
    def __init__(self, env):
